# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import os

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

city_data_df = city_data_df.dropna(subset=['Country'])
# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,dryden,49.8600,-93.5115,5.61,55,3,3.05,CA,1713146956
1,1,nova vicosa,24.7773,9.8506,17.90,11,100,2.91,LY,1713146957
2,2,bredasdorp,65.1555,-147.8868,0.29,79,100,3.29,US,1713146958
3,3,isafjordur,26.9263,-113.0168,20.18,39,93,9.14,MX,1713146958
4,4,chonchi,65.3386,-110.8638,-6.94,94,100,4.81,CA,1713146960


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng", "Lat", geo = True, tiles ="OSM", 
    size="Humidity", color="City", 
    width = 650,
    ylim = (90, -90)
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_city_df = city_data_df.loc[(city_data_df["Max Temp"] < 27) & 
(city_data_df["Max Temp"] > 21) & 
(city_data_df["Wind Speed"] < 4.5) & 
(city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values
narrowed_city_df = narrowed_city_df.dropna()
# Display sample data
narrowed_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
55,55,christchurch,40.9679,-98.3360,22.82,34,0,4.12,US,1713147456
127,127,shevchenko,38.4186,-92.6553,24.31,79,0,3.01,US,1713147489
162,162,port isabel,-28.4759,137.4473,26.63,18,0,3.18,AU,1713147508
183,183,whakatane,32.3763,-86.5337,22.10,72,0,2.06,US,1713147517
284,284,apolinario saravia,37.9284,-85.4665,23.59,37,0,3.09,US,1713147569
361,361,road town,34.8635,110.0526,23.30,22,0,1.89,CN,1713147611
369,369,empalme escobedo,30.6204,-90.4331,21.01,62,0,2.06,US,1713147616
377,377,nehe,36.7187,-87.2602,22.89,43,0,2.57,US,1713147620


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrowed_city_df[["City", "Country", "Lat", "Lng"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Hotel Name
55,christchurch,US,40.9679,-98.3360,
127,shevchenko,US,38.4186,-92.6553,
162,port isabel,AU,-28.4759,137.4473,
183,whakatane,US,32.3763,-86.5337,
284,apolinario saravia,US,37.9284,-85.4665,
361,road town,CN,34.8635,110.0526,
369,empalme escobedo,US,30.6204,-90.4331,
377,nehe,US,36.7187,-87.2602,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "filter": "",
    "bias" : "",
    "categories" : "accommodation.hotel",
    "apiKey" : geoapify_key,
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places?"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
christchurch - nearest hotel: Days Inn by Wyndham Grand Island
shevchenko - nearest hotel: Ozark Village Resort & Marina
port isabel - nearest hotel: No hotel found
whakatane - nearest hotel: Baymont
apolinario saravia - nearest hotel: Fairfield Inn & Suites
road town - nearest hotel: Huang He Hotel
empalme escobedo - nearest hotel: Bienvenue Mon Ami Bed & Breakfast
nehe - nearest hotel: Sleep Inn & Suites


,City,Country,Lat,Lng,Hotel Name
55,christchurch,US,40.9679,-98.3360,Days Inn by Wyndham Grand Island
127,shevchenko,US,38.4186,-92.6553,Ozark Village Resort & Marina
162,port isabel,AU,-28.4759,137.4473,No hotel found
183,whakatane,US,32.3763,-86.5337,Baymont
284,apolinario saravia,US,37.9284,-85.4665,Fairfield Inn & Suites
361,road town,CN,34.8635,110.0526,Huang He Hotel
369,empalme escobedo,US,30.6204,-90.4331,Bienvenue Mon Ami Bed & Breakfast
377,nehe,US,36.7187,-87.2602,Sleep Inn & Suites


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [12]:
%%capture --no-display

map_plot = hotel_df.hvplot.points(
    "Lng", "Lat",geo = True, tiles ="OSM", 
    hover_cols=['Hotel Name', 'Country'],
    color="City", 
    width = 650,
    ylim = (90, -90)
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)